In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "Suchae/test-finetuned"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_4bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
c:\Anaconda3\envs\kollm\Lib\site-packages\accelerate\utils\modeling.py:1389: UserWarning: Current model requires 1024.0 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
# EOS_TOKEN은 문장의 끝을 나타내는 토큰입니다. 이 토큰을 추가해야 합니다.
EOS_TOKEN = tokenizer.eos_token

# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = "<|end_of_text|>"  # end_token으로 사용할 토큰을 설정합니다.
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.


# 패딩 토큰을 eos_token으로 설정합니다.
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [6]:
# 바뀐 모델
from transformers import TextStreamer

# 입력 텍스트를 설정합니다.
input_text = alpaca_prompt.format(
    "테디노트 유튜브 채널에 대해서 알려주세요.",  # 지시사항
    "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
)
inputs = tokenizer([input_text], return_tensors="pt", padding=True).to('cuda')

# 생성된 텍스트를 스트리밍할 TextStreamer를 설정합니다.
text_streamer = TextStreamer(tokenizer)

# 원본 모델의 답변 생성
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다. 필요시 직접 정의할 수 있습니다.
)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
테디노트 유튜브 채널에 대해서 알려주세요.

### Response:
The Teddy Notes YouTube channel is a popular educational platform that provides engaging and informative content for children. The channel's main goal is to help young learners develop their language skills and build confidence in reading and writing. The channel's content is divided into three main categories: storytelling, vocabulary, and grammar. The storytelling section features animated videos that teach children about various topics, such as animals, food, and emotions. The vocabulary section focuses on teaching children common vocabulary words and phrases. The grammar section provides lessons on basic grammar rules and sentence structure. The channel's content is designed to be fun and 

KeyboardInterrupt: 

In [7]:
# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

In [8]:
# 파인튜닝된 모델의 답변 생성
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=4096,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다. 필요시 직접 정의할 수 있습니다.
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


interactive<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
테디노트 유튜브 채널에 대해서 알려주세요.

### Response:
테디노트(Teddy Notes)는 유튜브 채널입니다. 테디노트 채널은 다양한 교육 콘텐츠를 제공하고 있습니다. 테디노트 채널의 콘텐츠는 주로 영어 교육 콘텐츠입니다. 테디노트 채널의 

KeyboardInterrupt: 